## cl -cloud fraction (high clouds and low clouds)

In [1]:
a = ''

if a:
    print('executes')

In [15]:
import xarray as xr
import os
import numpy as np
import matplotlib.pyplot as plt

In [25]:
# folder = '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/historical/mon/atmos/Amon/r1i1p1/v20120227/cl'
# fileName = 'cl_Amon_GFDL-CM3_historical_r1i1p1_197001-197412.nc'
# path = folder + '/' + fileName

# ds = xr.open_dataset(path)
# ds

<xarray.Dataset>
Dimensions:     (time: 60, lat: 90, bnds: 2, lon: 144, lev: 48)
Coordinates:
  * lat         (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon         (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lev         (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time        (time) object 1970-01-16 12:00:00 ... 1974-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables: (12/14)
    average_DT  (time) timedelta64[ns] ...
    average_T1  (time) datetime64[ns] ...
    average_T2  (time) datetime64[ns] ...
    lat_bnds    (lat, bnds) float64 ...
    lon_bnds    (lon, bnds) float64 ...
    ps          (time, lat, lon) float32 ...
    ...          ...
    lev_bnds    (lev, bnds) float64 ...
    a           (lev) float64 ...
    a_bnds      (lev, bnds) float64 ...
    b           (lev) float64 ...
    b_bnds      (lev, bnds) float64 ...
    p0          float32 ...
Attributes: (12/27)
    title:                  NOAA GFDL GFDL-CM3, historical (run 1) experiment...
    institute_id:           NOAA GFDL
    source:                 GFDL-CM3 2010 atmosphere: AM3 (AM3p9,C48L48); sea...
    contact:                gfdl.climate.model.info@noaa.gov
    project_id:             CMIP5
    table_id:               Table Amon (31 Jan 2011)
    ...                     ...
    parent_experiment_id:   piControl
    physics_version:        1
    product:                output1
    institution:            NOAA GFDL(201 Forrestal Rd, Princeton, NJ, 08540)
    history:                File was processed by fremetar (GFDL analog of CM...
    parent_experiment_rip:  r1i1p1

In [33]:
folder = '/g/data/al33/replicas/CMIP5/combined/NOAA-GFDL/GFDL-CM3/rcp85/mon/atmos/Amon/r1i1p1/v20120227/cl'
fileName = 'cl_Amon_GFDL-CM3_rcp85_r1i1p1_207101-207512.nc'
path = folder + '/' + fileName

ds = xr.open_dataset(path)
ds

<xarray.Dataset>
Dimensions:     (time: 60, lat: 90, bnds: 2, lon: 144, lev: 48)
Coordinates:
  * lat         (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon         (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lev         (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time        (time) object 2071-01-16 12:00:00 ... 2075-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables: (12/14)
    average_DT  (time) timedelta64[ns] ...
    average_T1  (time) datetime64[ns] ...
    average_T2  (time) datetime64[ns] ...
    lat_bnds    (lat, bnds) float64 ...
    lon_bnds    (lon, bnds) float64 ...
    ps          (time, lat, lon) float32 ...
    ...          ...
    lev_bnds    (lev, bnds) float64 ...
    a           (lev) float64 ...
    a_bnds      (lev, bnds) float64 ...
    b           (lev) float64 ...
    b_bnds      (lev, bnds) float64 ...
    p0          float32 ...
Attributes: (12/27)
    title:                  NOAA GFDL GFDL-CM3, RCP8.5 (run 1) experiment out...
    institute_id:           NOAA GFDL
    source:                 GFDL-CM3 2010 atmosphere: AM3 (AM3p9,C48L48); sea...
    contact:                gfdl.climate.model.info@noaa.gov
    project_id:             CMIP5
    table_id:               Table Amon (31 Jan 2011)
    ...                     ...
    parent_experiment_id:   historical
    physics_version:        1
    product:                output1
    institution:            NOAA GFDL(201 Forrestal Rd, Princeton, NJ, 08540)
    history:                File was processed by fremetar (GFDL analog of CM...
    parent_experiment_rip:  r1i1p1

In [17]:
import intake
import scipy
import xarray as xr
import xesmf as xe



def regrid_conserv(ds_in, haveDsOut, path='/g/data/k10/cb4968/data/cmip5/FGOALS-g2/FGOALS-g2_ds_regid_historical.nc', modelDsOut='FGOALS-g2'):

    if haveDsOut:
        ds_out = xr.open_dataset(path)
        regrid = xe.Regridder(ds_in, ds_out, 'conservative', periodic=True)
    
    else:
        ds_dict = intake.cat.nci['esgf'].cmip5.search(
                                        model_id = modelDsOut, 
                                        experiment = 'historical',
                                        time_frequency = 'day', 
                                        realm = 'atmos', 
                                        ensemble = 'r1i1p1', 
                                        variable= 'pr').to_dataset_dict()

        ds_regrid = ds_dict[list(ds_dict.keys())[-1]].sel(time='1970-01-01', lon=slice(0,360),lat=slice(-30,30))
        ds_regrid.to_netcdf(path)

        ds_out = xr.open_dataset(path)
        regrid = xe.Regridder(ds_in, ds_out, 'conservative', periodic=True)
        
    return regrid(ds_in)

In [17]:
# ds_orig = ds
# haveDsOut = True
# ds_regrid = regrid_conserv(ds_orig, haveDsOut) # path='', model ='')

In [27]:
# fileList = [
#     'cl_Amon_GFDL-CM3_historical_r1i1p1_197001-197412.nc',
#     'cl_Amon_GFDL-CM3_historical_r1i1p1_197501-197912.nc',
#     'cl_Amon_GFDL-CM3_historical_r1i1p1_198001-198412.nc',
#     'cl_Amon_GFDL-CM3_historical_r1i1p1_198501-198912.nc',
#     'cl_Amon_GFDL-CM3_historical_r1i1p1_199001-199412.nc',
#     'cl_Amon_GFDL-CM3_historical_r1i1p1_199501-199912.nc',
# ]


In [34]:
fileList = [
    'cl_Amon_GFDL-CM3_rcp85_r1i1p1_207101-207512.nc',
    'cl_Amon_GFDL-CM3_rcp85_r1i1p1_207601-208012.nc',
    'cl_Amon_GFDL-CM3_rcp85_r1i1p1_208101-208512.nc',
    'cl_Amon_GFDL-CM3_rcp85_r1i1p1_208601-209012.nc',
    'cl_Amon_GFDL-CM3_rcp85_r1i1p1_209101-209512.nc',
    'cl_Amon_GFDL-CM3_rcp85_r1i1p1_209601-210012.nc'
]


In [35]:
path_full = []
for file in fileList:
    path_full = np.append(path_full, os.path.join(folder, file))

In [36]:
ds_all = xr.open_mfdataset(path_full, combine='by_coords').sel(lat=slice(-35,35))
ds_all

<xarray.Dataset>
Dimensions:     (time: 360, lat: 36, bnds: 2, lon: 144, lev: 48)
Coordinates:
  * lat         (lat) float64 -35.0 -33.0 -31.0 -29.0 ... 29.0 31.0 33.0 35.0
  * lon         (lon) float64 1.25 3.75 6.25 8.75 ... 351.2 353.8 356.2 358.8
  * lev         (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time        (time) object 2071-01-16 12:00:00 ... 2100-12-16 12:00:00
Dimensions without coordinates: bnds
Data variables: (12/14)
    average_DT  (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2  (time) datetime64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    lat_bnds    (time, lat, bnds) float64 dask.array<chunksize=(60, 36, 2), meta=np.ndarray>
    lon_bnds    (time, lon, bnds) float64 dask.array<chunksize=(60, 144, 2), meta=np.ndarray>
    ps          (time, lat, lon) float32 dask.array<chunksize=(60, 36, 144), meta=np.ndarray>
    ...          ...
    lev_bnds    (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    a           (time, lev) float64 dask.array<chunksize=(60, 48), meta=np.ndarray>
    a_bnds      (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    b           (time, lev) float64 dask.array<chunksize=(60, 48), meta=np.ndarray>
    b_bnds      (time, lev, bnds) float64 dask.array<chunksize=(60, 48, 2), meta=np.ndarray>
    p0          (time) float32 1.013e+05 1.013e+05 ... 1.013e+05 1.013e+05
Attributes: (12/27)
    title:                  NOAA GFDL GFDL-CM3, RCP8.5 (run 1) experiment out...
    institute_id:           NOAA GFDL
    source:                 GFDL-CM3 2010 atmosphere: AM3 (AM3p9,C48L48); sea...
    contact:                gfdl.climate.model.info@noaa.gov
    project_id:             CMIP5
    table_id:               Table Amon (31 Jan 2011)
    ...                     ...
    parent_experiment_id:   historical
    physics_version:        1
    product:                output1
    institution:            NOAA GFDL(201 Forrestal Rd, Princeton, NJ, 08540)
    history:                File was processed by fremetar (GFDL analog of CM...
    parent_experiment_rip:  r1i1p1

In [37]:
cl = ds_all.cl
cl

<xarray.DataArray 'cl' (time: 360, lev: 48, lat: 36, lon: 144)>
dask.array<getitem, shape=(360, 48, 36, 144), dtype=float32, chunksize=(60, 48, 36, 144), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -35.0 -33.0 -31.0 -29.0 -27.0 ... 29.0 31.0 33.0 35.0
  * lon      (lon) float64 1.25 3.75 6.25 8.75 11.25 ... 351.2 353.8 356.2 358.8
  * lev      (lev) float64 0.9961 0.9874 0.9767 ... 3.801e-05 1.711e-05
  * time     (time) object 2071-01-16 12:00:00 ... 2100-12-16 12:00:00
Attributes:
    cell_methods:      time: mean
    interp_method:     conserve_order1
    long_name:         Cloud Area Fraction
    original_name:     tot_cloud_area
    units:             %
    standard_name:     cloud_area_fraction_in_atmosphere_layer
    original_units:    percent
    cell_measures:     area: areacella
    associated_files:  baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation...

In [23]:
def save_file(dataSet, folder, fileName):
    
    os.makedirs(folder, exist_ok=True)
    path = folder + '/' + fileName

    if os.path.exists(path):
        os.remove(path)    
    
    dataSet.to_netcdf(path)

In [32]:
folder = '/g/data/k10/cb4968/data/cmip5/ds'
fileName = 'GFDL-CM3_cl_rcp85.nc'
save_file(cl, folder, fileName)